# Import development libraries

In [1]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import numpy as np

Ecoinvent and base data already imported

In [2]:
bd.projects.set_current("ecoinvent 3.7.1")

In [3]:
tox = ('ReCiPe Midpoint (E) V1.13', 'human toxicity', 'HTPinf')

2.5 gets rid of the `mapping` dictionary, using instead the SQLite row ids. It exposes this value with the `.id` property.

In [4]:
[(o.id, o.key) for o in bd.Database("ecoinvent 3.7.1") if o['name'] == 'transport, passenger car, electric']

[(7301, ('ecoinvent 3.7.1', '6f67d7bb34034ed6aef5f33536ae7781'))]

New calling convention for creating LCA objects, as we need more flexibility, and the ability to run `bw2calc` with just data files and without a `bw2data` database.

In [5]:
fu, data_objs, _ = bd.prepare_lca_inputs({('ecoinvent 3.7.1', '6f67d7bb34034ed6aef5f33536ae7781'): 1}, method=tox)

See [bw_processing](https://github.com/brightway-lca/bw_processing) for a description of these data packages.

In [6]:
data_objs

[ReadZipFS(PosixPath('/Users/cmutel/Library/Application Support/Brightway3/ecoinvent-371.040a8b7bfd29ab08dd0a24a6d8383a3d/processed/biosphere3.5d405d71.zip')),
 ReadZipFS(PosixPath('/Users/cmutel/Library/Application Support/Brightway3/ecoinvent-371.040a8b7bfd29ab08dd0a24a6d8383a3d/processed/ecoinvent-371.040a8b7b.zip')),
 ReadZipFS(PosixPath('/Users/cmutel/Library/Application Support/Brightway3/ecoinvent-371.040a8b7bfd29ab08dd0a24a6d8383a3d/processed/recipe-midpoint-e-v113hh.fd527ef1ee26c3a929822f74365a4036.zip'))]

In [7]:
lca = bc.LCA(fu, data_objs=data_objs)

In [8]:
lca.lci()
lca.lcia()

In [9]:
lca.score

7.987990671102498

# Modifying the supply chain

Let's test the impact of knowing specifically where our copper comes from

In [10]:
[(o.id, o.key) for o in bd.Database("ecoinvent 3.7.1") if o['name'] == 'market for copper concentrate, sulfide ore']

[(6498, ('ecoinvent 3.7.1', '00c41d5e9a1364c22ef9091479a65bbf'))]

In [11]:
copper = bd.get_activity(('ecoinvent 3.7.1', '00c41d5e9a1364c22ef9091479a65bbf'))
copper

'market for copper concentrate, sulfide ore' (kilogram, GLO, None)

This is a market, with different mines (and their varying technologies, concentrations, and energy sources) over the world contributing.

In [12]:
sum(exc['amount'] for exc in copper.technosphere())

1.000000000000001

The tox impact per kilogram of copper varies a lot, though some of this is due to allocation across multiple metals.

In [28]:
for exc in copper.technosphere():
    lca.redo_lcia({exc.input.id: 1})
    print(lca.score, exc.input)

655.7145722086279 'copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None)
2589.9396543988964 'copper mine operation and beneficiation, sulfide ore' (kilogram, CA, None)
487.10582138916044 'copper mine operation and beneficiation, sulfide ore' (kilogram, CL, None)
445.0130143211017 'copper mine operation and beneficiation, sulfide ore' (kilogram, CN, None)
1311.8694814528128 'copper mine operation and beneficiation, sulfide ore' (kilogram, ID, None)
172.04622200980634 'copper mine operation and beneficiation, sulfide ore' (kilogram, KZ, None)
611.7249552534782 'copper mine operation and beneficiation, sulfide ore' (kilogram, RU, None)
1186.829820670955 'copper mine operation and beneficiation, sulfide ore' (kilogram, US, None)
809.607986694068 'copper mine operation and beneficiation, sulfide ore' (kilogram, ZM, None)
600.3206987624394 'gold-silver mine operation and beneficiation' (kilogram, CA-QC, None)
1419.743025038946 'molybdenite mine operation' (kilogram, GLO,

# Modification approach 1: Create new LCA for each possibility

In [13]:
possibles = [exc.input for exc in copper.technosphere()]

In [15]:
possibles

['copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, CA, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, CL, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, CN, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, ID, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, KZ, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, RU, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, US, None),
 'copper mine operation and beneficiation, sulfide ore' (kilogram, ZM, None),
 'gold-silver mine operation and beneficiation' (kilogram, CA-QC, None),
 'molybdenite mine operation' (kilogram, GLO, None),
 'smelting and refining of nickel concentrate, 16% Ni' (kilogram, GLO, None),
 'zinc mine operation' (kilogram, GLO, None),
 'primary zinc production from concentrate' (kil

Our approach here is simple: We want to set the inputs to the market to zero to everything except the one input we are considering:

In [29]:
def create_replacement_vector_dp(parent, possibles, selected):
    modified = bwp.create_datapackage(sum_intra_duplicates=True, sum_inter_duplicates=False)
    # Everything is zero...
    data = np.zeros(len(possibles))
    indices = np.zeros(len(possibles), dtype=bwp.INDICES_DTYPE)
    
    for index, obj in enumerate(possibles):
        if obj.id == selected:
            # ... except the one input we selected. Minus sign because it is consumed.
            # Could also be positive, and then use a `flip` vector. See 
            # bw_processing and matrix_utils for more details.
            data[index] = -1
        indices[index] = (obj.id, parent)
        
    modified.add_persistent_vector(
        matrix="technosphere_matrix",
        indices_array=indices,
        name="Substitute global copper mix",
        data_array=data,
    )
    return modified

To use this new substituting data package, we just add it to the list of other data packages (but at the end, so it has the final word).

In [30]:
for supplier in possibles:
    dp = create_replacement_vector_dp(copper.id, possibles, supplier.id)
    
    lca = bc.LCA(fu, data_objs=data_objs + [dp])
    lca.lci()
    lca.lcia()
    
    print(lca.score, supplier)

7.7555770700264794 'copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None)
11.14631929710827 'copper mine operation and beneficiation, sulfide ore' (kilogram, CA, None)
7.460194029583274 'copper mine operation and beneficiation, sulfide ore' (kilogram, CL, None)
7.389906894877194 'copper mine operation and beneficiation, sulfide ore' (kilogram, CN, None)
8.902843777110897 'copper mine operation and beneficiation, sulfide ore' (kilogram, ID, None)
6.914925434803388 'copper mine operation and beneficiation, sulfide ore' (kilogram, KZ, None)
7.679308583364266 'copper mine operation and beneficiation, sulfide ore' (kilogram, RU, None)
8.6838008875667 'copper mine operation and beneficiation, sulfide ore' (kilogram, US, None)
8.026016270400383 'copper mine operation and beneficiation, sulfide ore' (kilogram, ZM, None)
7.659161249183263 'gold-silver mine operation and beneficiation' (kilogram, CA-QC, None)
9.091221629458627 'molybdenite mine operation' (kilogram, GLO, Non

# Modification approach 2: Array of possible suppliers

We can reuse the LCA object (though the technosphere matrix will be automatically regenerated) we use arrays instead of vectors.

This is an evolution of the [presamples](https://github.com/PascalLesage/presamples/) approach. One big change versus `presamples` is that we no can add whatever rows, columns, or values we want to the matrix, without needing to create dummy values to be modified later.

In [18]:
def create_replacement_array_dp(parent, possibles):
    modified = bwp.create_datapackage(sum_intra_duplicates=True, sum_inter_duplicates=False)
    data = np.diag(np.ones(len(possibles))) * -1
    indices = np.zeros(len(possibles), dtype=bwp.INDICES_DTYPE)
    
    for index, obj in enumerate(possibles):
        indices[index] = (obj.id, parent)
                
    modified.add_persistent_array(
        matrix="technosphere_matrix",
        indices_array=indices,
        name="Substitute global copper mix",
        data_array=data,
    )
    return modified

Arrays lets us express multiple scenarios or system configurations in one data format. Each column is a different configuration.

In [19]:
np.diag(np.ones(10)) * -1

array([[-1., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -1., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -1., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -1., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -1., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -1., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -1., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -1., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -1., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -1.]])

When using these configurations, we can just iterate through the LCA object with `next`. The iteration order for each data package is controlled by its [policies](https://github.com/brightway-lca/bw_processing#policies).

In [20]:
lca = bc.LCA(fu, data_objs=data_objs + [create_replacement_array_dp(copper.id, possibles)], use_arrays=True)
lca.lci()
lca.lcia()

for supplier in possibles:
    print(lca.score, supplier)
    next(lca)

6.706969655839139 'copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None)
7.7555770700264794 'copper mine operation and beneficiation, sulfide ore' (kilogram, CA, None)
7.659161249183263 'copper mine operation and beneficiation, sulfide ore' (kilogram, CL, None)
8.026016270400383 'copper mine operation and beneficiation, sulfide ore' (kilogram, CN, None)
7.7555770700264794 'copper mine operation and beneficiation, sulfide ore' (kilogram, ID, None)
7.460194029583274 'copper mine operation and beneficiation, sulfide ore' (kilogram, KZ, None)
9.091221629458627 'copper mine operation and beneficiation, sulfide ore' (kilogram, RU, None)
8.026016270400383 'copper mine operation and beneficiation, sulfide ore' (kilogram, US, None)
6.9821295490803115 'copper mine operation and beneficiation, sulfide ore' (kilogram, ZM, None)
9.091221629458627 'gold-silver mine operation and beneficiation' (kilogram, CA-QC, None)
8.902843777110897 'molybdenite mine operation' (kilogram, GLO,

The correct approach for you will depend on your problem, how imperatirve or functional you are feeling that day, your mood, etc ;)

See the other notebooks for more possibilities introduced in Brightway 2.5.